# 🏪 Комплексный анализ розничной сети магазинов

**Цель:** Исследование жизненного цикла магазинов, расчёт экономических и финансовых показателей

**Данные:** 20 магазинов, 10 товарных категорий, 2019-2026 (84 месяца)

---

In [6]:
# ============================================================
# НАСТРОЙКА ОКРУЖЕНИЯ
# ============================================================
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Настройки отображения
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

print("✓ Библиотеки загружены")

✓ Библиотеки загружены


In [ ]:
# ============================================================
# ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ
# ============================================================
df = pd.read_excel('final_flat_clean.xlsx')

# Словарь месяцев
month_map = {
    "Январь": 1, "Февраль": 2, "Март": 3, "Апрель": 4, "Май": 5, "Июнь": 6,
    "Июль": 7, "Август": 8, "Сентябрь": 9, "Октябрь": 10, "Ноябрь": 11, "Декабрь": 12
}

# Создаём дату
df['Month_Num'] = df['Месяц'].map(month_map)
df['Дата'] = pd.to_datetime(dict(year=df['Год'], month=df['Month_Num'], day=1))

# Сортировка магазинов по номеру
df['Store_ID'] = df['Магазин'].str.extract(r'(\d+)').astype(int)
df = df.sort_values(['Store_ID', 'Дата', 'Товар']).reset_index(drop=True)

# ============================================================
# РАСЧЁТ БАЗОВЫХ МЕТРИК
# ============================================================
# Выручка = Число чеков × Сумма в чеке
df['Выручка'] = df['Число чеков'] * df['Сумма в чеке']

# Валовая прибыль = Число чеков × Наценка
df['Валовая_прибыль'] = df['Число чеков'] * df['Наценка продажи в чеке']

# Маржинальность %
df['Маржа_%'] = (df['Наценка продажи в чеке'] / df['Сумма в чеке'] * 100).round(2)

print(f"✓ Данные загружены: {len(df):,} строк")
print(f"  Магазинов: {df['Магазин'].nunique()}")
print(f"  Категорий: {df['Товар'].nunique()}")
print(f"  Период: {df['Дата'].min().strftime('%Y-%m')} — {df['Дата'].max().strftime('%Y-%m')}")
df.head(3)

---
# 📊 БЛОК 1: ЖИЗНЕННЫЙ ЦИКЛ МАГАЗИНА

Анализ фаз развития: **Запуск → Рост → Зрелость → Насыщение → Спад**

---

In [ ]:
# ============================================================
# 1.1 АГРЕГАЦИЯ ПО МАГАЗИНАМ И МЕСЯЦАМ
# ============================================================
# Схлопываем товары — получаем общую выручку магазина за месяц
df_store = df.groupby(['Магазин', 'Store_ID', 'Дата']).agg({
    'Выручка': 'sum',
    'Валовая_прибыль': 'sum',
    'Число чеков': 'sum',
    'Количество в чеке': 'mean',
    'Сумма в чеке': 'mean',
    'Маржа_%': 'mean'
}).reset_index()

df_store = df_store.sort_values(['Store_ID', 'Дата'])

# Средний чек (пересчёт на агрегированном уровне)
df_store['Средний_чек'] = df_store['Выручка'] / df_store['Число чеков']

print(f"✓ Агрегировано: {len(df_store)} записей (магазин × месяц)")
df_store.head()

In [ ]:
# ============================================================
# 1.2 РАСЧЁТ ТРЕНДОВ И ПОКАЗАТЕЛЕЙ ЖИЗНЕННОГО ЦИКЛА
# ============================================================

def calculate_lifecycle_metrics(group):
    """Расчёт метрик жизненного цикла для одного магазина"""
    group = group.sort_values('Дата').copy()
    
    # 1. Скользящее среднее (тренд) — 6 месяцев
    group['Тренд_6м'] = group['Выручка'].rolling(window=6, min_periods=1).mean()
    
    # 2. Скользящее среднее — 12 месяцев (годовой тренд)
    group['Тренд_12м'] = group['Выручка'].rolling(window=12, min_periods=1).mean()
    
    # 3. Темп роста месяц к месяцу (MoM)
    group['MoM_%'] = group['Выручка'].pct_change() * 100
    
    # 4. Темп роста год к году (YoY)
    group['YoY_%'] = group['Выручка'].pct_change(periods=12) * 100
    
    # 5. Ускорение (вторая производная) — для определения точек перегиба
    group['Ускорение'] = group['Тренд_6м'].diff().diff()
    
    # 6. Нормализованный индекс (% от пика)
    max_revenue = group['Тренд_6м'].max()
    group['Индекс_от_пика'] = (group['Тренд_6м'] / max_revenue * 100).round(1)
    
    # 7. Волатильность (скользящее стандартное отклонение)
    group['Волатильность'] = group['Выручка'].rolling(window=6, min_periods=3).std()
    
    return group

# Применяем ко всем магазинам
df_lifecycle = df_store.groupby('Магазин', group_keys=False).apply(calculate_lifecycle_metrics)

print("✓ Метрики жизненного цикла рассчитаны")
df_lifecycle[['Магазин', 'Дата', 'Выручка', 'Тренд_6м', 'YoY_%', 'Индекс_от_пика']].head(15)

In [ ]:
# ============================================================
# 1.3 ОПРЕДЕЛЕНИЕ ФАЗЫ ЖИЗНЕННОГО ЦИКЛА
# ============================================================

def determine_phase(row):
    """
    Определение фазы жизненного цикла на основе:
    - YoY% (темп роста год к году)
    - Индекс от пика (насколько далеко от максимума)
    - Ускорение (замедляется или ускоряется рост)
    """
    yoy = row['YoY_%']
    idx = row['Индекс_от_пика']
    accel = row['Ускорение']
    
    # Обработка NaN
    if pd.isna(yoy):
        return 'Запуск'  # Первый год — нет данных для сравнения
    
    # Логика определения фазы
    if yoy > 15:  # Быстрый рост
        return 'Рост'
    elif yoy > 5:  # Умеренный рост
        if idx > 90:
            return 'Зрелость'
        else:
            return 'Рост'
    elif yoy > -5:  # Стабильность (±5%)
        if idx > 85:
            return 'Зрелость'
        else:
            return 'Насыщение'
    elif yoy > -15:  # Умеренное падение
        return 'Насыщение'
    else:  # Сильное падение
        return 'Спад'

df_lifecycle['Фаза'] = df_lifecycle.apply(determine_phase, axis=1)

# Статистика по фазам
phase_stats = df_lifecycle.groupby('Фаза').agg({
    'Выручка': ['mean', 'std'],
    'YoY_%': 'mean',
    'Магазин': 'count'
}).round(0)

print("📊 Распределение по фазам жизненного цикла:")
print(df_lifecycle['Фаза'].value_counts())
print("\n")
phase_stats

In [ ]:
# ============================================================
# 1.4 ВИЗУАЛИЗАЦИЯ: HEATMAP ЖИЗНЕННОГО ЦИКЛА
# ============================================================

# Создаём pivot для heatmap
pivot_lifecycle = df_lifecycle.pivot(index='Магазин', columns='Дата', values='Индекс_от_пика')

# Сортируем магазины по номеру
store_order = [f'Магазин {i}' for i in range(1, 21)]
pivot_lifecycle = pivot_lifecycle.reindex(store_order)

fig = px.imshow(
    pivot_lifecycle,
    labels=dict(x="Период", y="Магазин", color="% от пика"),
    aspect="auto",
    color_continuous_scale="RdYlGn",  # Красный-Жёлтый-Зелёный
    title="🔥 Жизненный цикл магазинов (Индекс от пика продаж)"
)

fig.update_layout(
    height=600,
    title_font_size=18,
    xaxis_title=None,
    coloraxis_colorbar=dict(title="% от пика", ticksuffix="%")
)

fig.update_traces(
    hovertemplate="<b>%{y}</b><br>Дата: %{x|%b %Y}<br>Индекс: %{z:.1f}%<extra></extra>"
)

fig.show()

In [ ]:
# ============================================================
# 1.5 ВИЗУАЛИЗАЦИЯ: ТРЕНДЫ ВЫРУЧКИ ПО МАГАЗИНАМ
# ============================================================

# Цветовая схема для фаз
phase_colors = {
    'Запуск': '#9467bd',     # Фиолетовый
    'Рост': '#2ca02c',       # Зелёный
    'Зрелость': '#1f77b4',   # Синий
    'Насыщение': '#ff7f0e',  # Оранжевый
    'Спад': '#d62728'        # Красный
}

fig = make_subplots(
    rows=5, cols=4,
    subplot_titles=[f'Магазин {i}' for i in range(1, 21)],
    vertical_spacing=0.08,
    horizontal_spacing=0.05
)

for i in range(1, 21):
    store_data = df_lifecycle[df_lifecycle['Store_ID'] == i].sort_values('Дата')
    row = (i - 1) // 4 + 1
    col = (i - 1) % 4 + 1
    
    # Линия выручки
    fig.add_trace(
        go.Scatter(
            x=store_data['Дата'],
            y=store_data['Выручка'],
            mode='lines',
            line=dict(color='lightgray', width=1),
            name='Факт',
            showlegend=False
        ),
        row=row, col=col
    )
    
    # Линия тренда
    fig.add_trace(
        go.Scatter(
            x=store_data['Дата'],
            y=store_data['Тренд_6м'],
            mode='lines',
            line=dict(color='#1f77b4', width=2),
            name='Тренд',
            showlegend=False
        ),
        row=row, col=col
    )

fig.update_layout(
    height=900,
    title_text="📈 Динамика выручки магазинов (серый: факт, синий: тренд 6м)",
    title_font_size=18,
    showlegend=False
)

fig.update_xaxes(tickformat="%Y", dtick="M24")
fig.update_yaxes(tickformat=",.0f")

fig.show()

In [ ]:
# ============================================================
# 1.6 КЛАСТЕРИЗАЦИЯ МАГАЗИНОВ ПО ТРАЕКТОРИИ
# ============================================================
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Создаём профиль каждого магазина
store_profiles = df_lifecycle.groupby('Магазин').agg({
    'Выручка': ['mean', 'std', 'min', 'max'],
    'YoY_%': 'mean',
    'Индекс_от_пика': ['mean', 'last'],
    'Волатильность': 'mean'
}).round(2)

store_profiles.columns = ['_'.join(col).strip() for col in store_profiles.columns]
store_profiles = store_profiles.reset_index()

# Признаки для кластеризации
features = ['Выручка_mean', 'Выручка_std', 'YoY_%_mean', 'Индекс_от_пика_last', 'Волатильность_mean']
X = store_profiles[features].fillna(0)

# Обработка бесконечных значений
X = X.replace([np.inf, -np.inf], np.nan)  # Заменяем бесконечности на NaN
X = X.fillna(0)  # Заменяем NaN на 0

# Стандартизация
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# K-Means кластеризация
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
store_profiles['Кластер'] = kmeans.fit_predict(X_scaled)

# Интерпретация кластеров
cluster_names = {
    0: 'Стабильные лидеры',
    1: 'Растущие звёзды',
    2: 'Проблемные',
    3: 'Середняки'
}

# Определяем названия на основе средних значений
cluster_stats = store_profiles.groupby('Кластер')[features].mean()
print("📊 Характеристики кластеров:")
display(cluster_stats.round(0))

print("\n🏪 Распределение магазинов по кластерам:")
print(store_profiles[['Магазин', 'Кластер', 'Выручка_mean', 'YoY_%_mean', 'Индекс_от_пика_last']].sort_values('Кластер'))

In [ ]:
# ============================================================
# 1.7 ВИЗУАЛИЗАЦИЯ КЛАСТЕРОВ
# ============================================================

fig = px.scatter(
    store_profiles,
    x='Выручка_mean',
    y='YoY_%_mean',
    size='Волатильность_mean',
    color='Кластер',
    text='Магазин',
    title='🎯 Кластеризация магазинов по эффективности',
    labels={
        'Выручка_mean': 'Средняя выручка (руб/мес)',
        'YoY_%_mean': 'Средний темп роста YoY (%)',
        'Волатильность_mean': 'Волатильность'
    },
    color_continuous_scale='Viridis'
)

fig.update_traces(textposition='top center', textfont_size=9)
fig.update_layout(height=600)

# Добавляем линии осей
fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)
fig.add_vline(x=store_profiles['Выручка_mean'].median(), line_dash="dash", line_color="gray", opacity=0.5)

fig.show()

In [ ]:
# ============================================================
# 1.8 СВОДНАЯ ТАБЛИЦА ЖИЗНЕННОГО ЦИКЛА
# ============================================================

# Текущая фаза для каждого магазина (последний месяц)
current_phase = df_lifecycle.sort_values('Дата').groupby('Магазин').last()[['Фаза', 'Индекс_от_пика', 'YoY_%']]

# Объединяем с профилями
lifecycle_summary = store_profiles.merge(current_phase, on='Магазин')

# Расчёт CAGR (среднегодовой темп роста)
def calc_cagr(store_name):
    data = df_lifecycle[df_lifecycle['Магазин'] == store_name].sort_values('Дата')
    first_year = data.groupby(data['Дата'].dt.year)['Выручка'].sum().iloc[0]
    last_year = data.groupby(data['Дата'].dt.year)['Выручка'].sum().iloc[-1]
    years = data['Дата'].dt.year.nunique() - 1
    if years > 0 and first_year > 0:
        return ((last_year / first_year) ** (1/years) - 1) * 100
    return 0

lifecycle_summary['CAGR_%'] = lifecycle_summary['Магазин'].apply(calc_cagr).round(1)

# Финальная сводка
final_summary = lifecycle_summary[[
    'Магазин', 'Кластер', 'Фаза', 'Выручка_mean', 'YoY_%', 'CAGR_%', 'Индекс_от_пика', 'Волатильность_mean'
]].sort_values('Выручка_mean', ascending=False)

final_summary.columns = ['Магазин', 'Кластер', 'Текущая фаза', 'Ср. выручка', 'YoY%', 'CAGR%', 'Индекс от пика', 'Волатильность']

print("📋 СВОДНАЯ ТАБЛИЦА ЖИЗНЕННОГО ЦИКЛА МАГАЗИНОВ")
print("="*80)
final_summary

---
# 📈 БЛОК 2: ОПЕРАЦИОННАЯ ЭФФЕКТИВНОСТЬ

Ключевые KPI магазина: выручка, трафик, средний чек, маржа, UPT

---

In [ ]:
# ============================================================
# 2.1 РАСЧЁТ ВСЕХ KPI НА УРОВНЕ МАГАЗИНА
# ============================================================

# Агрегация по магазинам (за весь период)
kpi_store = df.groupby('Магазин').agg({
    'Выручка': 'sum',
    'Валовая_прибыль': 'sum',
    'Число чеков': 'sum',
    'Количество в чеке': 'mean',
    'Сумма в чеке': 'mean',
    'Наценка продажи в чеке': 'mean',
    'Маржа_%': 'mean',
    'Store_ID': 'first'
}).reset_index()

# Расчёт производных KPI
kpi_store['Средний_чек'] = kpi_store['Выручка'] / kpi_store['Число чеков']
kpi_store['UPT'] = kpi_store['Количество в чеке']  # Units per Transaction
kpi_store['Gross_Margin_%'] = (kpi_store['Валовая_прибыль'] / kpi_store['Выручка'] * 100).round(2)
kpi_store['Прибыль_на_чек'] = kpi_store['Валовая_прибыль'] / kpi_store['Число чеков']

# Индексы относительно среднего по сети
kpi_store['Индекс_выручки'] = (kpi_store['Выручка'] / kpi_store['Выручка'].mean() * 100).round(1)
kpi_store['Индекс_трафика'] = (kpi_store['Число чеков'] / kpi_store['Число чеков'].mean() * 100).round(1)
kpi_store['Индекс_чека'] = (kpi_store['Средний_чек'] / kpi_store['Средний_чек'].mean() * 100).round(1)
kpi_store['Индекс_маржи'] = (kpi_store['Gross_Margin_%'] / kpi_store['Gross_Margin_%'].mean() * 100).round(1)

kpi_store = kpi_store.sort_values('Store_ID')

print("📊 KPI МАГАЗИНОВ (за весь период)")
print("="*100)
kpi_store[['Магазин', 'Выручка', 'Валовая_прибыль', 'Число чеков', 'Средний_чек', 'UPT', 'Gross_Margin_%']].round(0)

In [ ]:
# ============================================================
# 2.2 ВИЗУАЛИЗАЦИЯ: СРАВНИТЕЛЬНАЯ ДИАГРАММА KPI
# ============================================================

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Выручка по магазинам', 
        'Валовая прибыль по магазинам',
        'Трафик (число чеков)',
        'Средний чек'
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

store_names = [f'М{i}' for i in range(1, 21)]

# 1. Выручка
fig.add_trace(
    go.Bar(x=store_names, y=kpi_store['Выручка'], marker_color='#1f77b4', name='Выручка'),
    row=1, col=1
)
fig.add_hline(y=kpi_store['Выручка'].mean(), line_dash="dash", line_color="red", row=1, col=1)

# 2. Валовая прибыль
fig.add_trace(
    go.Bar(x=store_names, y=kpi_store['Валовая_прибыль'], marker_color='#2ca02c', name='Прибыль'),
    row=1, col=2
)
fig.add_hline(y=kpi_store['Валовая_прибыль'].mean(), line_dash="dash", line_color="red", row=1, col=2)

# 3. Трафик
fig.add_trace(
    go.Bar(x=store_names, y=kpi_store['Число чеков'], marker_color='#ff7f0e', name='Чеки'),
    row=2, col=1
)
fig.add_hline(y=kpi_store['Число чеков'].mean(), line_dash="dash", line_color="red", row=2, col=1)

# 4. Средний чек
fig.add_trace(
    go.Bar(x=store_names, y=kpi_store['Средний_чек'], marker_color='#9467bd', name='Ср.чек'),
    row=2, col=2
)
fig.add_hline(y=kpi_store['Средний_чек'].mean(), line_dash="dash", line_color="red", row=2, col=2)

fig.update_layout(
    height=600,
    title_text="📊 Сравнение KPI магазинов (красная линия = среднее по сети)",
    showlegend=False
)

fig.show()

In [ ]:
# ============================================================
# 2.3 ДИНАМИКА KPI ВО ВРЕМЕНИ
# ============================================================

# Агрегация по месяцам (вся сеть)
kpi_monthly = df.groupby('Дата').agg({
    'Выручка': 'sum',
    'Валовая_прибыль': 'sum',
    'Число чеков': 'sum',
    'Маржа_%': 'mean'
}).reset_index()

kpi_monthly['Средний_чек'] = kpi_monthly['Выручка'] / kpi_monthly['Число чеков']
kpi_monthly['Gross_Margin_%'] = kpi_monthly['Валовая_прибыль'] / kpi_monthly['Выручка'] * 100

# Скользящие средние для трендов
kpi_monthly['Выручка_тренд'] = kpi_monthly['Выручка'].rolling(6, min_periods=1).mean()
kpi_monthly['Чек_тренд'] = kpi_monthly['Средний_чек'].rolling(6, min_periods=1).mean()
kpi_monthly['Трафик_тренд'] = kpi_monthly['Число чеков'].rolling(6, min_periods=1).mean()

# Темпы роста YoY
kpi_monthly['Выручка_YoY'] = kpi_monthly['Выручка'].pct_change(12) * 100
kpi_monthly['Трафик_YoY'] = kpi_monthly['Число чеков'].pct_change(12) * 100
kpi_monthly['Чек_YoY'] = kpi_monthly['Средний_чек'].pct_change(12) * 100

print("📈 Динамика KPI сети по месяцам")
kpi_monthly[['Дата', 'Выручка', 'Число чеков', 'Средний_чек', 'Gross_Margin_%', 'Выручка_YoY']].tail(12).round(1)

In [ ]:
# ============================================================
# 2.4 ВИЗУАЛИЗАЦИЯ: ДИНАМИКА KPI СЕТИ
# ============================================================

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Выручка сети (млн руб)', 
        'Трафик сети (тыс. чеков)',
        'Средний чек (руб)',
        'Валовая маржа (%)'
    ],
    specs=[[{"secondary_y": True}, {"secondary_y": True}],
           [{"secondary_y": True}, {"secondary_y": True}]]
)

# 1. Выручка
fig.add_trace(
    go.Scatter(x=kpi_monthly['Дата'], y=kpi_monthly['Выручка']/1e6, 
               mode='lines', line=dict(color='lightblue', width=1), name='Факт'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=kpi_monthly['Дата'], y=kpi_monthly['Выручка_тренд']/1e6, 
               mode='lines', line=dict(color='blue', width=2), name='Тренд'),
    row=1, col=1
)

# 2. Трафик
fig.add_trace(
    go.Scatter(x=kpi_monthly['Дата'], y=kpi_monthly['Число чеков']/1e3, 
               mode='lines', line=dict(color='lightsalmon', width=1), name='Факт'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=kpi_monthly['Дата'], y=kpi_monthly['Трафик_тренд']/1e3, 
               mode='lines', line=dict(color='orange', width=2), name='Тренд'),
    row=1, col=2
)

# 3. Средний чек
fig.add_trace(
    go.Scatter(x=kpi_monthly['Дата'], y=kpi_monthly['Средний_чек'], 
               mode='lines', line=dict(color='lightgreen', width=1), name='Факт'),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=kpi_monthly['Дата'], y=kpi_monthly['Чек_тренд'], 
               mode='lines', line=dict(color='green', width=2), name='Тренд'),
    row=2, col=1
)

# 4. Маржа
fig.add_trace(
    go.Scatter(x=kpi_monthly['Дата'], y=kpi_monthly['Gross_Margin_%'], 
               mode='lines', line=dict(color='purple', width=2), name='Маржа'),
    row=2, col=2
)

fig.update_layout(
    height=600,
    title_text="📈 Динамика ключевых KPI сети во времени",
    showlegend=False
)

fig.show()

In [ ]:
# ============================================================
# 2.5 СЕЗОННОСТЬ: ИНДЕКСЫ ПО МЕСЯЦАМ
# ============================================================

# Расчёт сезонных индексов
seasonality = df.groupby('Месяц').agg({
    'Выручка': 'mean',
    'Число чеков': 'mean',
    'Сумма в чеке': 'mean'
}).reset_index()

# Нормируем относительно среднего
seasonality['Индекс_выручки'] = (seasonality['Выручка'] / seasonality['Выручка'].mean() * 100).round(1)
seasonality['Индекс_трафика'] = (seasonality['Число чеков'] / seasonality['Число чеков'].mean() * 100).round(1)
seasonality['Индекс_чека'] = (seasonality['Сумма в чеке'] / seasonality['Сумма в чеке'].mean() * 100).round(1)

# Порядок месяцев
month_order = ["Январь", "Февраль", "Март", "Апрель", "Май", "Июнь",
               "Июль", "Август", "Сентябрь", "Октябрь", "Ноябрь", "Декабрь"]
seasonality['Месяц'] = pd.Categorical(seasonality['Месяц'], categories=month_order, ordered=True)
seasonality = seasonality.sort_values('Месяц')

# Визуализация
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=seasonality['Индекс_выручки'],
    theta=seasonality['Месяц'],
    fill='toself',
    name='Выручка',
    line_color='blue'
))

fig.add_trace(go.Scatterpolar(
    r=seasonality['Индекс_трафика'],
    theta=seasonality['Месяц'],
    fill='toself',
    name='Трафик',
    line_color='orange'
))

fig.add_trace(go.Scatterpolar(
    r=seasonality['Индекс_чека'],
    theta=seasonality['Месяц'],
    fill='toself',
    name='Средний чек',
    line_color='green'
))

fig.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[80, 120])),
    title="🗓️ Сезонность продаж (индекс 100 = среднее)",
    height=500
)

fig.show()

print("\n📊 Сезонные индексы:")
seasonality[['Месяц', 'Индекс_выручки', 'Индекс_трафика', 'Индекс_чека']]

In [ ]:
# ============================================================
# 2.6 СВОДНАЯ ТАБЛИЦА ЭФФЕКТИВНОСТИ МАГАЗИНОВ
# ============================================================

# Рейтинг магазинов по комплексному показателю
kpi_store['Рейтинг_выручки'] = kpi_store['Выручка'].rank(ascending=False).astype(int)
kpi_store['Рейтинг_маржи'] = kpi_store['Gross_Margin_%'].rank(ascending=False).astype(int)
kpi_store['Рейтинг_трафика'] = kpi_store['Число чеков'].rank(ascending=False).astype(int)
kpi_store['Рейтинг_чека'] = kpi_store['Средний_чек'].rank(ascending=False).astype(int)

# Комплексный рейтинг (среднее место)
kpi_store['Комплексный_рейтинг'] = (
    kpi_store['Рейтинг_выручки'] + 
    kpi_store['Рейтинг_маржи'] + 
    kpi_store['Рейтинг_трафика'] + 
    kpi_store['Рейтинг_чека']
) / 4

# Определение статуса
def get_status(rank):
    if rank <= 5:
        return '🟢 Лидер'
    elif rank <= 10:
        return '🟡 Выше среднего'
    elif rank <= 15:
        return '🟠 Ниже среднего'
    else:
        return '🔴 Аутсайдер'

kpi_store['Статус'] = kpi_store['Комплексный_рейтинг'].apply(get_status)

# Финальная таблица
efficiency_table = kpi_store[[
    'Магазин', 'Статус', 'Комплексный_рейтинг',
    'Выручка', 'Рейтинг_выручки',
    'Gross_Margin_%', 'Рейтинг_маржи',
    'Число чеков', 'Рейтинг_трафика',
    'Средний_чек', 'Рейтинг_чека'
]].sort_values('Комплексный_рейтинг')

print("📋 РЕЙТИНГ ЭФФЕКТИВНОСТИ МАГАЗИНОВ")
print("="*100)
efficiency_table.round(1)

---
# 🔬 БЛОК 3: ПРОДВИНУТАЯ АНАЛИТИКА

- Анализ по типам товаров (Скоропорт / Товар / Непрод)
- ABC/XYZ анализ
- Декомпозиция выручки
- Когортный анализ категорий

---

In [ ]:
# ============================================================
# 3.1 АНАЛИЗ ПО ТИПАМ ТОВАРОВ
# ============================================================

# Проверяем наличие колонки Тип
if 'Тип' not in df.columns:
    print("⚠️ Колонка 'Тип' не найдена. Перезапустите Untitled.ipynb для обновления данных.")
else:
    print(f"✓ Найдены типы товаров: {df['Тип'].unique()}")
    
    # Агрегация по типам
    type_analysis = df.groupby('Тип').agg({
        'Выручка': 'sum',
        'Валовая_прибыль': 'sum',
        'Число чеков': 'sum',
        'Маржа_%': 'mean'
    }).reset_index()
    
    type_analysis['Средний_чек'] = type_analysis['Выручка'] / type_analysis['Число чеков']
    type_analysis['Gross_Margin_%'] = (type_analysis['Валовая_прибыль'] / type_analysis['Выручка'] * 100).round(2)
    type_analysis['Доля_выручки_%'] = (type_analysis['Выручка'] / type_analysis['Выручка'].sum() * 100).round(1)
    type_analysis['Доля_прибыли_%'] = (type_analysis['Валовая_прибыль'] / type_analysis['Валовая_прибыль'].sum() * 100).round(1)
    
    print("\n📊 АНАЛИЗ ПО ТИПАМ ТОВАРОВ")
    print("="*80)
    display(type_analysis.round(0))

In [ ]:
# ============================================================
# 3.2 ВИЗУАЛИЗАЦИЯ: СТРУКТУРА ПО ТИПАМ ТОВАРОВ
# ============================================================

if 'Тип' in df.columns:
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=['Доля в выручке', 'Доля в прибыли', 'Маржинальность'],
        specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "bar"}]]
    )
    
    colors = {'Скоропорт': '#ff6b6b', 'Товар': '#4ecdc4', 'Непрод': '#45b7d1'}
    color_list = [colors.get(t, '#808080') for t in type_analysis['Тип']]
    
    # Pie 1: Выручка
    fig.add_trace(
        go.Pie(labels=type_analysis['Тип'], values=type_analysis['Выручка'],
               marker=dict(colors=color_list), textinfo='label+percent'),
        row=1, col=1
    )
    
    # Pie 2: Прибыль
    fig.add_trace(
        go.Pie(labels=type_analysis['Тип'], values=type_analysis['Валовая_прибыль'],
               marker=dict(colors=color_list), textinfo='label+percent'),
        row=1, col=2
    )
    
    # Bar: Маржа
    fig.add_trace(
        go.Bar(x=type_analysis['Тип'], y=type_analysis['Gross_Margin_%'],
               marker=dict(color=color_list), text=type_analysis['Gross_Margin_%'].round(1),
               textposition='outside'),
        row=1, col=3
    )
    
    fig.update_layout(
        height=400,
        title_text="📦 Структура продаж по типам товаров",
        showlegend=False
    )
    
    fig.show()

In [ ]:
# ============================================================
# 3.3 ТИПЫ ТОВАРОВ ПО МАГАЗИНАМ (HEATMAP)
# ============================================================

if 'Тип' in df.columns:
    # Pivot: магазин × тип → выручка
    type_by_store = df.groupby(['Магазин', 'Тип'])['Выручка'].sum().unstack(fill_value=0)
    
    # Нормализация по строкам (доля каждого типа в магазине)
    type_by_store_pct = type_by_store.div(type_by_store.sum(axis=1), axis=0) * 100
    
    # Сортировка магазинов
    store_order = [f'Магазин {i}' for i in range(1, 21)]
    type_by_store_pct = type_by_store_pct.reindex(store_order)
    
    fig = px.imshow(
        type_by_store_pct,
        labels=dict(x="Тип товара", y="Магазин", color="Доля %"),
        aspect="auto",
        color_continuous_scale="Blues",
        title="🏪 Структура типов товаров по магазинам (%)"
    )
    
    fig.update_layout(height=600)
    fig.update_traces(
        text=type_by_store_pct.round(1).values,
        texttemplate="%{text}%",
        hovertemplate="<b>%{y}</b><br>%{x}: %{z:.1f}%<extra></extra>"
    )
    
    fig.show()
    
    print("\n📊 Доля типов товаров по магазинам (%):")
    display(type_by_store_pct.round(1))

In [ ]:
# ============================================================
# 3.4 ДИНАМИКА ТИПОВ ТОВАРОВ ВО ВРЕМЕНИ
# ============================================================

if 'Тип' in df.columns:
    # Агрегация по дате и типу
    type_dynamics = df.groupby(['Дата', 'Тип']).agg({
        'Выручка': 'sum',
        'Валовая_прибыль': 'sum'
    }).reset_index()
    
    # Stacked area chart
    fig = px.area(
        type_dynamics,
        x='Дата',
        y='Выручка',
        color='Тип',
        title='📈 Динамика выручки по типам товаров',
        color_discrete_map={'Скоропорт': '#ff6b6b', 'Товар': '#4ecdc4', 'Непрод': '#45b7d1'}
    )
    
    fig.update_layout(height=400)
    fig.show()
    
    # Линейный график долей
    type_pivot = type_dynamics.pivot(index='Дата', columns='Тип', values='Выручка')
    type_share = type_pivot.div(type_pivot.sum(axis=1), axis=0) * 100
    
    fig2 = go.Figure()
    for col in type_share.columns:
        fig2.add_trace(go.Scatter(
            x=type_share.index, y=type_share[col],
            mode='lines', name=col, stackgroup='one'
        ))
    
    fig2.update_layout(
        title='📊 Изменение долей типов товаров во времени (%)',
        height=400,
        yaxis_title='Доля %'
    )
    fig2.show()

In [ ]:
# ============================================================
# 3.5 ABC-АНАЛИЗ ТОВАРНЫХ КАТЕГОРИЙ
# ============================================================

# ABC по выручке
abc_data = df.groupby('Товар').agg({
    'Выручка': 'sum',
    'Валовая_прибыль': 'sum',
    'Число чеков': 'sum'
}).reset_index()

abc_data = abc_data.sort_values('Выручка', ascending=False)
abc_data['Выручка_накоп'] = abc_data['Выручка'].cumsum()
abc_data['Доля_накоп_%'] = (abc_data['Выручка_накоп'] / abc_data['Выручка'].sum() * 100).round(1)
abc_data['Доля_%'] = (abc_data['Выручка'] / abc_data['Выручка'].sum() * 100).round(1)

# Присвоение ABC-класса
def assign_abc(cumsum_pct):
    if cumsum_pct <= 80:
        return 'A'
    elif cumsum_pct <= 95:
        return 'B'
    else:
        return 'C'

abc_data['ABC'] = abc_data['Доля_накоп_%'].apply(assign_abc)

# Маржинальность
abc_data['Маржа_%'] = (abc_data['Валовая_прибыль'] / abc_data['Выручка'] * 100).round(1)

print("📊 ABC-АНАЛИЗ ТОВАРНЫХ КАТЕГОРИЙ")
print("="*80)
print("A = 80% выручки | B = 15% выручки | C = 5% выручки")
print("-"*80)
display(abc_data[['Товар', 'ABC', 'Выручка', 'Доля_%', 'Доля_накоп_%', 'Маржа_%']])

In [ ]:
# ============================================================
# 3.6 XYZ-АНАЛИЗ (СТАБИЛЬНОСТЬ СПРОСА)
# ============================================================

# Расчёт коэффициента вариации по месяцам для каждого товара
xyz_data = df.groupby(['Товар', 'Дата'])['Выручка'].sum().reset_index()
xyz_stats = xyz_data.groupby('Товар')['Выручка'].agg(['mean', 'std']).reset_index()
xyz_stats['CV_%'] = (xyz_stats['std'] / xyz_stats['mean'] * 100).round(1)

# Присвоение XYZ-класса
def assign_xyz(cv):
    if cv <= 10:
        return 'X'  # Стабильный спрос
    elif cv <= 25:
        return 'Y'  # Сезонный спрос
    else:
        return 'Z'  # Хаотичный спрос

xyz_stats['XYZ'] = xyz_stats['CV_%'].apply(assign_xyz)

print("📊 XYZ-АНАЛИЗ (стабильность спроса)")
print("="*80)
print("X = CV ≤ 10% (стабильный) | Y = CV ≤ 25% (сезонный) | Z = CV > 25% (хаотичный)")
print("-"*80)
display(xyz_stats[['Товар', 'XYZ', 'mean', 'std', 'CV_%']].rename(
    columns={'mean': 'Ср. выручка/мес', 'std': 'Стд. отклонение'}
).round(0))

In [ ]:
# ============================================================
# 3.7 МАТРИЦА ABC-XYZ
# ============================================================

# Объединяем ABC и XYZ
abc_xyz = abc_data[['Товар', 'ABC', 'Выручка', 'Доля_%', 'Маржа_%']].merge(
    xyz_stats[['Товар', 'XYZ', 'CV_%']], on='Товар'
)
abc_xyz['Группа'] = abc_xyz['ABC'] + abc_xyz['XYZ']

# Добавляем тип товара если есть
if 'Тип' in df.columns:
    type_mapping = df.groupby('Товар')['Тип'].first().reset_index()
    abc_xyz = abc_xyz.merge(type_mapping, on='Товар')

print("📊 МАТРИЦА ABC-XYZ")
print("="*80)
print("""
Рекомендации по группам:
• AX, BX — Ключевые товары, точный прогноз, минимум запасов
• AY, BY — Сезонные хиты, планировать по сезону
• AZ, BZ — Непредсказуемые лидеры, держать страховой запас
• CX, CY, CZ — Кандидаты на вывод или сокращение ассортимента
""")
print("-"*80)

# Сортируем по важности группы
group_order = ['AX', 'AY', 'AZ', 'BX', 'BY', 'BZ', 'CX', 'CY', 'CZ']
abc_xyz['Группа_сорт'] = pd.Categorical(abc_xyz['Группа'], categories=group_order, ordered=True)
abc_xyz = abc_xyz.sort_values('Группа_сорт')

display(abc_xyz.drop(columns=['Группа_сорт']))

In [ ]:
# ============================================================
# 3.8 ДЕКОМПОЗИЦИЯ ИЗМЕНЕНИЯ ВЫРУЧКИ (Revenue Bridge)
# ============================================================

# Сравниваем первый и последний год
first_year = df['Год'].min()
last_year = df['Год'].max()

# Агрегация по годам
yearly = df.groupby('Год').agg({
    'Выручка': 'sum',
    'Число чеков': 'sum',
    'Сумма в чеке': 'mean'
}).reset_index()

yearly['Средний_чек'] = yearly['Выручка'] / yearly['Число чеков']

# Берём первый и последний год
y1 = yearly[yearly['Год'] == first_year].iloc[0]
y2 = yearly[yearly['Год'] == last_year].iloc[0]

# Декомпозиция изменения выручки
# ΔВыручка = ΔТрафик × Базовый_чек + Базовый_трафик × ΔЧек + ΔТрафик × ΔЧек
delta_traffic = y2['Число чеков'] - y1['Число чеков']
delta_check = y2['Средний_чек'] - y1['Средний_чек']

# Эффект трафика (при старом чеке)
traffic_effect = delta_traffic * y1['Средний_чек']

# Эффект чека (при старом трафике)
check_effect = y1['Число чеков'] * delta_check

# Комбинированный эффект
combined_effect = delta_traffic * delta_check

# Общее изменение
total_change = y2['Выручка'] - y1['Выручка']

print(f"📊 ДЕКОМПОЗИЦИЯ ИЗМЕНЕНИЯ ВЫРУЧКИ ({first_year} → {last_year})")
print("="*80)
print(f"\n{'Показатель':<30} {first_year:>15} {last_year:>15} {'Δ':>15}")
print("-"*80)
print(f"{'Выручка':<30} {y1['Выручка']:>15,.0f} {y2['Выручка']:>15,.0f} {total_change:>+15,.0f}")
print(f"{'Трафик (чеков)':<30} {y1['Число чеков']:>15,.0f} {y2['Число чеков']:>15,.0f} {delta_traffic:>+15,.0f}")
print(f"{'Средний чек':<30} {y1['Средний_чек']:>15,.0f} {y2['Средний_чек']:>15,.0f} {delta_check:>+15,.0f}")

print(f"\n{'ДЕКОМПОЗИЦИЯ ИЗМЕНЕНИЯ:':<50}")
print("-"*80)
print(f"{'Эффект трафика (ΔТрафик × Старый чек)':<50} {traffic_effect:>+15,.0f}")
print(f"{'Эффект чека (Старый трафик × ΔЧек)':<50} {check_effect:>+15,.0f}")
print(f"{'Комбинированный эффект (ΔТрафик × ΔЧек)':<50} {combined_effect:>+15,.0f}")
print("-"*80)
print(f"{'ИТОГО изменение выручки':<50} {total_change:>+15,.0f}")

In [ ]:
# ============================================================
# 3.9 КОГОРТНЫЙ АНАЛИЗ КАТЕГОРИЙ ПО ГОДАМ
# ============================================================

# Pivot: год × товар → выручка
cohort_data = df.groupby(['Год', 'Товар'])['Выручка'].sum().unstack()

# Темпы роста по годам
cohort_growth = cohort_data.pct_change() * 100

# Heatmap роста
fig = px.imshow(
    cohort_growth.T,
    labels=dict(x="Год", y="Товар", color="Рост %"),
    aspect="auto",
    color_continuous_scale="RdYlGn",
    color_continuous_midpoint=0,
    title="📊 Когортный анализ: темпы роста категорий по годам (%)"
)

fig.update_layout(height=500)
fig.update_traces(
    text=cohort_growth.T.round(0).values,
    texttemplate="%{text}%",
    hovertemplate="<b>%{y}</b><br>Год: %{x}<br>Рост: %{z:.1f}%<extra></extra>"
)

fig.show()

# Сводка по годам
print("\n📊 Выручка по категориям и годам (млн руб):")
display((cohort_data / 1e6).round(1))

---
# 📐 БЛОК 4: АНАЛИЗ ОПТИМАЛЬНОЙ ПЛОЩАДИ МАГАЗИНА

**4 метода определения оптимальной площади:**

1. **Производительность на м²** — максимизация выручки/м² и прибыли/м²
2. **Нелинейная регрессия** — поиск точки убывающей отдачи
3. **Предельная эффективность** — маржинальный анализ (∂Revenue/∂Area)
4. **Кластеризация + бенчмаркинг** — лидеры по размерным сегментам

---

In [3]:
# ============================================================
# 4.0 ЗАГРУЗКА ДАННЫХ О ПЛОЩАДИ
# ============================================================

print("="*80)
print("📐 БЛОК 4: АНАЛИЗ ОПТИМАЛЬНОЙ ПЛОЩАДИ МАГАЗИНА")
print("="*80)

STORE_AREA_FILE = "store.xlsx"

try:
    df_area = pd.read_excel(STORE_AREA_FILE)
    col_names = df_area.columns.tolist()
    print(f"✓ Загружен файл: {STORE_AREA_FILE}")
    print(f"  Колонки: {col_names}")

    df_area = df_area.rename(columns={
        "Магазин": "Магазин_адрес",
        "Торговая площадь магазина": "Площадь_м2"
    })

    print("Первые строки файла:")
    display(df_area.head())

except FileNotFoundError:
    print(f"❌ ОШИБКА: Файл {STORE_AREA_FILE} не найден!")
    raise

In [7]:
# ============================================================
# 4.1 ОБЪЕДИНЕНИЕ ДАННЫХ
# ============================================================

# Загрузка основных данных продаж
df = pd.read_excel("final_flat_clean.xlsx")

# Рассчитываем метрики
df["Выручка"] = df["Сумма в чеке"]
df["Валовая_прибыль"] = df["Наценка продажи в чеке"]
df["Store_ID"] = df["Магазин"].str.extract(r"(\d+)").astype(float)

# Агрегируем продажи по магазинам (за весь период)
store_summary = df.groupby("Магазин").agg({
    "Выручка": "sum",
    "Валовая_прибыль": "sum",
    "Число чеков": "sum",
    "Store_ID": "first"
}).reset_index()

store_summary["Маржа_%"] = store_summary["Валовая_прибыль"] / store_summary["Выручка"] * 100

# Пытаемся смержить по названию магазина
df_area["Магазин"] = df_area["Магазин_адрес"]

# Мерж данных
store_with_area = store_summary.merge(
    df_area[["Магазин", "Площадь_м2"]],
    on="Магазин",
    how="left"
)

# Проверка успешности мержа
missing_area = store_with_area["Площадь_м2"].isna().sum()
if missing_area > 0:
    print(f"⚠️ ВНИМАНИЕ: Для {missing_area} магазинов не найдена площадь!")
    print("Магазины без площади:")
    print(store_with_area[store_with_area["Площадь_м2"].isna()]["Магазин"].tolist())
    print("Доступные адреса в файле площадей:")
    print(df_area["Магазин_адрес"].tolist())

    # Попытка автоматического сопоставления по номеру
    print("🔧 Попытка автоматического сопоставления...")
    store_with_area["temp_id"] = store_with_area["Магазин"].str.extract(r"(\d+)").astype(float)
    df_area["temp_id"] = df_area["Магазин_адрес"].str.extract(r"(\d+)").astype(float)

    store_with_area = store_with_area.drop(columns=["Площадь_м2"], errors="ignore")
    store_with_area = store_with_area.merge(
        df_area[["temp_id", "Площадь_м2"]],
        on="temp_id",
        how="left"
    )
    store_with_area = store_with_area.drop(columns=["temp_id"], errors="ignore")
    df_area = df_area.drop(columns=["temp_id"], errors="ignore")

    missing_after = store_with_area["Площадь_м2"].isna().sum()
    if missing_after == 0:
        print("✓ Автоматическое сопоставление успешно!")
    else:
        print(f"❌ Остались магазины без площади: {missing_after}")

# Удаляем магазины без площади
store_with_area = store_with_area.dropna(subset=["Площадь_м2"])

print(f"✓ Объединение завершено: {len(store_with_area)} магазинов с площадью")
print(f"  Средняя площадь: {store_with_area['Площадь_м2'].mean():.1f} м²")
print(f"  Диапазон: {store_with_area['Площадь_м2'].min():.1f} - {store_with_area['Площадь_м2'].max():.1f} м²")

# Расчёт базовых метрик
store_with_area["Выручка_на_м2"] = store_with_area["Выручка"] / store_with_area["Площадь_м2"]
store_with_area["Прибыль_на_м2"] = store_with_area["Валовая_прибыль"] / store_with_area["Площадь_м2"]
store_with_area["Трафик_на_м2"] = store_with_area["Число чеков"] / store_with_area["Площадь_м2"]

display(store_with_area.sort_values("Store_ID").round(0))

In [ ]:
# ============================================================
# МЕТОД 1: АНАЛИЗ ПРОИЗВОДИТЕЛЬНОСТИ НА М²
# ============================================================

print("\n" + "="*80)
print("📊 МЕТОД 1: АНАЛИЗ ПРОИЗВОДИТЕЛЬНОСТИ НА М²")
print("="*80)
print("Цель: Найти площадь с максимальной выручкой/м² и прибылью/м²\n")

# Сортировка по эффективности
top_by_revenue_m2 = store_with_area.nlargest(5, 'Выручка_на_м2')[['Магазин', 'Площадь_м2', 'Выручка_на_м2', 'Прибыль_на_м2']]
top_by_profit_m2 = store_with_area.nlargest(5, 'Прибыль_на_м2')[['Магазин', 'Площадь_м2', 'Прибыль_на_м2', 'Выручка_на_м2']]

print("🏆 ТОП-5 по выручке на м²:")
display(top_by_revenue_m2.round(0))

print("\n🏆 ТОП-5 по прибыли на м²:")
display(top_by_profit_m2.round(0))

# Оптимальная площадь = средняя площадь лидеров
optimal_area_method1_revenue = top_by_revenue_m2['Площадь_м2'].mean()
optimal_area_method1_profit = top_by_profit_m2['Площадь_м2'].mean()

print(f"\n💡 ВЫВОД МЕТОДА 1:")
print(f"  • Оптимальная площадь (по выручке/м²): {optimal_area_method1_revenue:.0f} м²")
print(f"  • Оптимальная площадь (по прибыли/м²): {optimal_area_method1_profit:.0f} м²")
print(f"  • Рекомендуемый диапазон: {top_by_revenue_m2['Площадь_м2'].min():.0f}-{top_by_revenue_m2['Площадь_м2'].max():.0f} м²")

# Визуализация
fig = px.scatter(
    store_with_area,
    x='Площадь_м2',
    y='Выручка_на_м2',
    size='Выручка',
    color='Прибыль_на_м2',
    text='Store_ID',
    title='📊 Метод 1: Зависимость выручки/м² от площади магазина',
    labels={'Площадь_м2': 'Площадь (м²)', 'Выручка_на_м2': 'Выручка на м² (руб)', 'Прибыль_на_м2': 'Прибыль/м²'},
    color_continuous_scale='RdYlGn'
)

fig.add_vline(x=optimal_area_method1_revenue, line_dash="dash", line_color="red",
              annotation_text=f"Оптимум: {optimal_area_method1_revenue:.0f} м²")

fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.show()

In [ ]:
# ============================================================
# МЕТОД 2: НЕЛИНЕЙНАЯ РЕГРЕССИЯ (ПОИСК ТОЧКИ ПЕРЕГИБА)
# ============================================================

print("\n" + "="*80)
print("📈 МЕТОД 2: НЕЛИНЕЙНАЯ РЕГРЕССИЯ")
print("="*80)
print("Цель: Найти точку убывающей отдачи (где дополнительная площадь даёт <10% прироста)\n")

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score

# Подготовка данных
X = store_with_area['Площадь_м2'].values.reshape(-1, 1)
y_revenue = store_with_area['Выручка'].values

# 1. Линейная модель
lr = LinearRegression()
lr.fit(X, y_revenue)
y_pred_linear = lr.predict(X)
r2_linear = r2_score(y_revenue, y_pred_linear)

# 2. Полиномиальная модель (2-й степени)
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)
lr_poly = LinearRegression()
lr_poly.fit(X_poly, y_revenue)
y_pred_poly = lr_poly.predict(X_poly)
r2_poly = r2_score(y_revenue, y_pred_poly)

# 3. Кубическая модель
poly3 = PolynomialFeatures(degree=3)
X_poly3 = poly3.fit_transform(X)
lr_poly3 = LinearRegression()
lr_poly3.fit(X_poly3, y_revenue)
y_pred_poly3 = lr_poly3.predict(X_poly3)
r2_poly3 = r2_score(y_revenue, y_pred_poly3)

print(f"R² моделей:")
print(f"  • Линейная: {r2_linear:.3f}")
print(f"  • Квадратичная: {r2_poly:.3f}")
print(f"  • Кубическая: {r2_poly3:.3f}")

# Поиск оптимума для квадратичной модели
if r2_poly > r2_linear and len(lr_poly.coef_) >= 3:
    a, b, c = lr_poly.coef_[0], lr_poly.coef_[1], lr_poly.coef_[2]
    if c < 0:
        optimal_area_method2 = -b / (2 * c)
        print(f"\n💡 ВЫВОД МЕТОДА 2:")
        print(f"  • Модель: Квадратичная (y = {a:.0f} + {b:.0f}*x + {c:.2f}*x²)")
        print(f"  • Оптимальная площадь (точка максимума): {optimal_area_method2:.0f} м²")

        areas = np.arange(X.min(), X.max(), 10)
        revenues = lr_poly.predict(poly.transform(areas.reshape(-1, 1)))
        marginal_revenue = np.gradient(revenues, areas)

        threshold_idx = np.where(marginal_revenue < marginal_revenue.max() * 0.5)[0]
        if len(threshold_idx) > 0:
            diminishing_point = areas[threshold_idx[0]]
            print(f"  • Точка убывающей отдачи (50% от макс): {diminishing_point:.0f} м²")
    else:
        optimal_area_method2 = X.mean()
        print(f"\n💡 ВЫВОД МЕТОДА 2:")
        print(f"  • Зависимость линейная, максимум не обнаружен")
        print(f"  • Рекомендуемая площадь (среднее): {optimal_area_method2:.0f} м²")
else:
    optimal_area_method2 = X.mean()
    print(f"\n💡 ВЫВОД МЕТОДА 2:")
    print(f"  • Лучшая модель: Линейная (постоянная отдача от площади)")
    print(f"  • Рекомендуемая площадь (среднее): {optimal_area_method2:.0f} м²")

# Визуализация
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=store_with_area['Площадь_м2'],
    y=store_with_area['Выручка'],
    mode='markers',
    name='Факт',
    marker=dict(size=10, color='blue')
))

X_sorted = np.sort(X.flatten())
fig.add_trace(go.Scatter(
    x=X_sorted,
    y=lr.predict(X_sorted.reshape(-1, 1)),
    mode='lines',
    name=f'Линейная (R²={r2_linear:.2f})',
    line=dict(dash='dash', color='green')
))

fig.add_trace(go.Scatter(
    x=X_sorted,
    y=lr_poly.predict(poly.transform(X_sorted.reshape(-1, 1))),
    mode='lines',
    name=f'Квадратичная (R²={r2_poly:.2f})',
    line=dict(color='red', width=3)
))

if 'optimal_area_method2' in locals():
    fig.add_vline(x=optimal_area_method2, line_dash="dot", line_color="orange",
                  annotation_text=f"Оптимум: {optimal_area_method2:.0f} м²")

fig.update_layout(
    title='📈 Метод 2: Регрессионный анализ выручки от площади',
    xaxis_title='Площадь (м²)',
    yaxis_title='Выручка (руб)',
    height=500
)
fig.show()

In [ ]:
# ============================================================
# МЕТОД 3: ПРЕДЕЛЬНАЯ ЭФФЕКТИВНОСТЬ (МАРЖИНАЛЬНЫЙ АНАЛИЗ)
# ============================================================

print("\n" + "="*80)
print("💹 МЕТОД 3: ПРЕДЕЛЬНАЯ ЭФФЕКТИВНОСТЬ")
print("="*80)
print("Цель: Найти площадь, где предельная выручка начинает падать\n")

# Сортируем магазины по площади
marginal_df = store_with_area.sort_values('Площадь_м2').copy()

# Расчёт предельной выручки
marginal_df['Delta_Area'] = marginal_df['Площадь_м2'].diff()
marginal_df['Delta_Revenue'] = marginal_df['Выручка'].diff()
marginal_df['Marginal_Revenue'] = marginal_df['Delta_Revenue'] / marginal_df['Delta_Area']

# Скользящее среднее для сглаживания
marginal_df['Marginal_Revenue_MA'] = marginal_df['Marginal_Revenue'].rolling(3, min_periods=1).mean()

# Находим точку пика
peak_idx = marginal_df['Marginal_Revenue_MA'].idxmax()
optimal_area_method3 = marginal_df.loc[peak_idx, 'Площадь_м2'] if not pd.isna(peak_idx) else marginal_df['Площадь_м2'].median()

print("📊 Предельная выручка по площади:")
print(marginal_df[['Магазин', 'Площадь_м2', 'Выручка', 'Marginal_Revenue', 'Marginal_Revenue_MA']].round(0))

print(f"\n💡 ВЫВОД МЕТОДА 3:")
print(f"  • Оптимальная площадь (пик предельной выручки): {optimal_area_method3:.0f} м²")
print(f"  • Макс. предельная выручка: {marginal_df['Marginal_Revenue_MA'].max():.0f} руб/м²")

# Визуализация
fig = make_subplots(rows=2, cols=1, subplot_titles=['Выручка от площади', 'Предельная выручка (∂Revenue/∂Area)'])

fig.add_trace(
    go.Scatter(x=marginal_df['Площадь_м2'], y=marginal_df['Выручка'],
               mode='lines+markers', name='Выручка'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=marginal_df['Площадь_м2'], y=marginal_df['Marginal_Revenue'],
               mode='markers', name='Факт', marker=dict(color='lightgray')),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=marginal_df['Площадь_м2'], y=marginal_df['Marginal_Revenue_MA'],
               mode='lines', name='Тренд (MA-3)', line=dict(color='red', width=3)),
    row=2, col=1
)

fig.add_vline(x=optimal_area_method3, line_dash="dash", line_color="green", row=2, col=1,
              annotation_text=f"Оптимум: {optimal_area_method3:.0f} м²")

fig.update_layout(height=700, title_text='💹 Метод 3: Маржинальный анализ')
fig.update_xaxes(title_text="Площадь (м²)")
fig.update_yaxes(title_text="Выручка (руб)", row=1, col=1)
fig.update_yaxes(title_text="Предельная выручка (руб/м²)", row=2, col=1)
fig.show()

In [ ]:
# ============================================================
# МЕТОД 4: КЛАСТЕРИЗАЦИЯ + БЕНЧМАРКИНГ
# ============================================================

print("\n" + "="*80)
print("🎯 МЕТОД 4: КЛАСТЕРИЗАЦИЯ ПО РАЗМЕРУ + БЕНЧМАРКИНГ")
print("="*80)
print("Цель: Разделить магазины на размерные кластеры и найти лидеров в каждом\n")

# Определяем кластеры по площади (квантили)
store_with_area['Размер_кластер'] = pd.qcut(
    store_with_area['Площадь_м2'],
    q=3,
    labels=['Малые', 'Средние', 'Крупные'],
    duplicates='drop'
)

# Анализ по кластерам
cluster_analysis = store_with_area.groupby('Размер_кластер').agg({
    'Площадь_м2': ['min', 'max', 'mean'],
    'Выручка_на_м2': 'mean',
    'Прибыль_на_м2': 'mean',
    'Магазин': 'count'
}).round(0)

cluster_analysis.columns = ['_'.join(col).strip() for col in cluster_analysis.columns]
print("📊 Характеристики кластеров:")
display(cluster_analysis)

# Находим лидеров в каждом кластере
print("\n🏆 Лидеры по кластерам (максимальная прибыль/м²):")
leaders = []
for cluster in store_with_area['Размер_кластер'].unique():
    cluster_data = store_with_area[store_with_area['Размер_кластер'] == cluster]
    leader = cluster_data.nlargest(1, 'Прибыль_на_м2').iloc[0]
    leaders.append(leader)
    print(f"\n{cluster}:")
    print(f"  Магазин: {leader['Магазин']}")
    print(f"  Площадь: {leader['Площадь_м2']:.0f} м²")
    print(f"  Выручка/м²: {leader['Выручка_на_м2']:.0f} руб")
    print(f"  Прибыль/м²: {leader['Прибыль_на_м2']:.0f} руб")

# Оптимальная площадь = средняя площадь лидеров
optimal_area_method4 = np.mean([l['Площадь_м2'] for l in leaders])

print(f"\n💡 ВЫВОД МЕТОДА 4:")
print(f"  • Оптимальная площадь (среднее лидеров): {optimal_area_method4:.0f} м²")
print(f"  • Рекомендации по кластерам:")
for cluster in store_with_area['Размер_кластер'].unique():
    cluster_data = store_with_area[store_with_area['Размер_кластер'] == cluster]
    leader = cluster_data.nlargest(1, 'Прибыль_на_м2').iloc[0]
    print(f"    - {cluster}: ориентир {leader['Площадь_м2']:.0f} м² (эталон: {leader['Магазин']})")

# Визуализация
fig = px.box(
    store_with_area,
    x='Размер_кластер',
    y='Выручка_на_м2',
    points='all',
    title='🎯 Метод 4: Эффективность по размерным кластерам',
    labels={'Размер_кластер': 'Кластер', 'Выручка_на_м2': 'Выручка на м² (руб)'},
    color='Размер_кластер'
)

for leader in leaders:
    fig.add_annotation(
        x=leader['Размер_кластер'],
        y=leader['Выручка_на_м2'],
        text=f"★ {leader['Площадь_м2']:.0f} м²",
        showarrow=True,
        arrowhead=2
    )

fig.update_layout(height=500)
fig.show()

In [ ]:
# ============================================================
# ИТОГОВАЯ СВОДКА ВСЕХ МЕТОДОВ
# ============================================================

print("\n" + "="*80)
print("📋 ИТОГОВАЯ СВОДКА: ОПТИМАЛЬНАЯ ПЛОЩАДЬ МАГАЗИНА")
print("="*80)

results = {
    'Метод': [
        '1. Производительность на м²',
        '2. Нелинейная регрессия',
        '3. Предельная эффективность',
        '4. Кластеризация + бенчмаркинг'
    ],
    'Оптимальная площадь (м²)': [
        optimal_area_method1_profit,
        optimal_area_method2,
        optimal_area_method3,
        optimal_area_method4
    ],
    'Обоснование': [
        'Средняя площадь топ-5 по прибыли/м²',
        'Точка максимума квадратичной функции',
        'Пик предельной выручки',
        'Средняя площадь лидеров по кластерам'
    ]
}

summary_df = pd.DataFrame(results)
display(summary_df)

# Итоговая рекомендация (медиана всех методов)
valid_areas = [a for a in results['Оптимальная площадь (м²)'] if a is not None]
final_optimal_area = np.median(valid_areas)
final_optimal_range = (np.percentile(valid_areas, 25), np.percentile(valid_areas, 75))

print(f"\n🎯 ФИНАЛЬНАЯ РЕКОМЕНДАЦИЯ:")
print(f"  • Оптимальная площадь: {final_optimal_area:.0f} м² (медиана всех методов)")
print(f"  • Рекомендуемый диапазон: {final_optimal_range[0]:.0f} - {final_optimal_range[1]:.0f} м² (25-75 перцентиль)")
print(f"\n📊 Статистика текущих магазинов:")
print(f"  • Средняя площадь: {store_with_area['Площадь_м2'].mean():.0f} м²")
print(f"  • Медианная площадь: {store_with_area['Площадь_м2'].median():.0f} м²")
print(f"  • Диапазон: {store_with_area['Площадь_м2'].min():.0f} - {store_with_area['Площадь_м2'].max():.0f} м²")

# Классификация магазинов
store_with_area['Статус_площади'] = store_with_area['Площадь_м2'].apply(
    lambda x: '🟢 Оптимальная' if final_optimal_range[0] <= x <= final_optimal_range[1]
              else ('🔴 Слишком малая' if x < final_optimal_range[0] else '🟠 Слишком большая')
)

print(f"\n🏪 Распределение магазинов по статусу:")
print(store_with_area['Статус_площади'].value_counts())

print(f"\n💡 РЕКОМЕНДАЦИИ ПО МАГАЗИНАМ:")
for status in ['🔴 Слишком малая', '🟠 Слишком большая']:
    stores = store_with_area[store_with_area['Статус_площади'] == status]
    if len(stores) > 0:
        print(f"\n{status}:")
        for _, store in stores.iterrows():
            delta = final_optimal_area - store['Площадь_м2']
            action = 'расширить' if delta > 0 else 'сократить'
            print(f"  • {store['Магазин']}: {store['Площадь_м2']:.0f} м² → {action} на {abs(delta):.0f} м²")

# Финальная визуализация
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=store_with_area['Площадь_м2'],
    y=store_with_area['Выручка_на_м2'],
    mode='markers+text',
    marker=dict(
        size=15,
        color=store_with_area['Прибыль_на_м2'],
        colorscale='RdYlGn',
        showscale=True,
        colorbar=dict(title="Прибыль/м²")
    ),
    text=store_with_area['Store_ID'],
    textposition='top center',
    name='Магазины',
    hovertemplate='<b>%{text}</b><br>Площадь: %{x:.0f} м²<br>Выручка/м²: %{y:,.0f} руб<extra></extra>'
))

fig.add_vrect(
    x0=final_optimal_range[0], x1=final_optimal_range[1],
    fillcolor="green", opacity=0.1,
    annotation_text="Оптимальный диапазон", annotation_position="top left"
)

fig.add_vline(x=final_optimal_area, line_dash="dash", line_color="red", line_width=3,
              annotation_text=f"Оптимум: {final_optimal_area:.0f} м²")

fig.update_layout(
    title='🎯 ИТОГОВАЯ КАРТА: Оптимальная площадь магазина',
    xaxis_title='Площадь (м²)',
    yaxis_title='Выручка на м² (руб)',
    height=600,
    showlegend=True
)

fig.show()

print("\n" + "="*80)
print("✓ АНАЛИЗ ЗАВЕРШЁН")
print("="*80)